# 💻 Interaktive kart

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GMGI221-2024/forelesninger/blob/main/11_interaktive_kart.ipynb)

[Online kart](https://link.springer.com/referenceworkentry/10.1007/978-3-319-23519-6_1485-2)
har vært interaktive i lang tid: nesten alle online kart tillater å zoome
inn og ut, å panorere i kartet, og å velge kartfunksjoner, eller ellers
forespørre informasjon om dem.

Interaktivt innhold i nettsider, som online kart, er typisk
implementert ved hjelp av
[*JavaScript*/*ECMAScript*](https://en.wikipedia.org/wiki/ECMAScript), et skriptspråk
opprinnelig rettet mot nettsider, primært, men brukt for mange andre
applikasjoner.

I det åpne kildekoderiket finnes det en rekke forskjellige *JavaScript*
biblioteker for interaktiv webkartografi, inkludert
[Leaflet](https://leafletjs.com/), som vi vil bruke i denne leksjonen, og
[OpenLayers](https://openlayers.org/).

Ta det rolig, vi vil ikke måtte skrive en eneste linje med *JavaScript*; dette er et
*Python* kurs, tross alt. Heller, vi vil dra nytte av
[*Folium*](https://python-visualization.github.io/folium/) Python-pakken: den
hjelper med å lage interaktive *Leaflet*-kart fra data lagret i
`geopandas.GeoDataFrame`s.


:::{admonition} *Folium* ressurser
:class: note

Finn mer informasjon om mulighetene til *Folium*-pakken på dens
offisielle nettsider:
- [Dokumentasjon](https://python-visualization.github.io/folium/)
- [Eksempelgalleri](https://nbviewer.org/github/python-visualization/folium/tree/main/examples/)
- [Hurtigstart guide](https://python-visualization.github.io/folium/quickstart.html#Getting-Started)
:::


## Opprette et enkelt interaktivt webkart

Vi vil starte med å lage et enkelt interaktivt webkart som ikke inneholder noe
annet enn et bakgrunnskart. Dette er for at vi skal bli vant til hvordan *Folium*’s syntaks fungerer, og
hvilke trinn vi må ta.

Vi lager et `folium.Map`-objekt, og spesifiserer rundt hvilken `location` det skal sentreres
og på hvilket utgangs-zoomnivå (~0-20) kartet skal vises. Ved å sette
`control_scale` til `True`, får vi *Folium* til å vise en skala også.

In [2]:
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

# Vi vil eksportere HTML-sider i løpet av denne leksjonen,
# la oss også forberede en utdatamappe for dem:


### Lagre det resulterende kartet

For å lagre dette kartet til en HTML-fil som kan åpnes i en hvilken som helst nettleser,
bruk [`folium.Map.save()`](https://python-visualization.github.io/branca/element.html#branca.element.Element.save):

### Endre bakgrunnskartet

Hvis du vil bruke et annet bakgrunnslag enn standard OpenStreetMap,
aksepterer `folium.Map` parameteret `tiles`, som kan referere til [en av de
innebygde kartleverandørene](https://python-visualization.github.io/folium/modules.html#folium.folium.Map).

Mens vi er i gang, la oss også variere senterlokasjonen og zoomnivået på kartet:

Eller vi kan peke på en egendefinert *tiles URL*:

In [ ]:
#tiles="https://mt1.google.com/vt/lyrs=r&x={x}&y={y}&z={z}",


## Legg til en punktmarkør

For å legge til en enkelt markør til et *Folium* kart, opprett en
[`folium.Marker`](https://python-visualization.github.io/folium/modules.html#folium.map.Marker).
Gi en
[`folium.Icon`](https://python-visualization.github.io/folium/modules.html#folium.map.Icon)
som parameter `icon` for å påvirke hvordan markøren er stylet, og sett `tooltip`
for å vise en tekst når musepekeren beveger seg over den.

## Legg til et lag med punkter

*Folium* støtter også å legge til hele lag, for eksempel, som
`geopandas.GeoDataFrames`. *Folium* implementerer [*Leaflet*'s `geoJSON`
lag](https://leafletjs.com/reference.html#geojson) i sin
`folium.features.GeoJson` klasse. Vi kan initialisere en slik klasse (og lag)
med en geodataframe, og legge den til et kart. I eksempelet nedenfor bruker vi
`adresser.gpkg` datasettet vi lager [i leksjon
3](#05_geokoding_i_geopandas).

Vi kan også legge til et popup-vindu på kartet vårt som viser adressene på interessepunktet ved å klikke:

## Legg til et polygonlag

I den følgende delen skal vi gjenbesøke et annet datasett som vi har jobbet med før: Befolkningsrutenettet for Oslo fra SSB.

:::{admonition} Indekskolonne for koroplettkart
:class: hint

Vi vil bruke `folium.Choropleth` for å vise befolkningsrutenettet. Koroplettkart
er mer enn bare polygon-geometrier, som kan vises som et
`folium.features.GeoJson`-lag, akkurat som vi brukte for adressene, ovenfor. Snarere tar klassen seg av kategorisering av data, legger til en legende, og
noen flere små oppgaver for å raskt lage temakart.

Klassen forventer et inputdatasett som har en eksplisitt, `str`-type, indeks
kolonne, da den behandler den geografiske inputen og den tematiske inputen som separate
datasett som må samles (se også, nedenfor, hvordan vi spesifiserer både
`geo_data` og `data`).

En god tilnærming for å lage en slik kolonne er å kopiere dataframens indeks
til en ny kolonne, for eksempel `id`.
:::

Nå kan vi lage koroplettlaget for polygoner, og legge det til et kartobjekt.
På grunn av den litt komplekse arkitekturen til *Folium*, må vi gi en
rekke parametere:
- `geo_data` og `data`, geografiske og tematiske inputdatasett,
  henholdsvis. Kan være den samme `geopandas.GeoDataFrame`en.
- `columns`: en tuple av navnene på relevante kolonner i `data`: en unik
  indekskolonne, og kolonnen som inneholder tematiske data
- `key_on`: hvilken kolonne i `geo_data` som skal brukes for å koble `data` (dette er
  i utgangspunktet identisk med `columns`, bortsett fra at det bare er den første verdien)

For å gjøre kartet litt finere, la oss fortsatt be om flere kategorier (`bins`),
endre fargeområdet (ved hjelp av `fill_color`), sett linjetykkelsen til null,
og legge til en tittel til legenden:

### Legg til en tooltip til et koroplettkart

I et slikt interaktivt kart, ville det være fint å vise verdien av hvert
rutenett-polygon når du holder musepekeren over det. *Folium* støtter ikke
dette out-of-the-box, men med et enkel triks kan vi utvide funksjonaliteten:
Vi legger til et gjennomsiktig polygonlag ved hjelp av en 'grunnleggende' `folium.features.GeoJson`,
og konfigurerer den til å vise tooltips.

Vi kan beholde `map` som vi laget ovenfor, og bare legge til et annet lag på det..

In [ ]:
# folium GeoJson lag forventer en stylingfunksjon,
# som mottar hver av kartets funksjon og returnerer
# en individuell stil. Den kan imidlertid også returnere en
# statisk stil:
def style_function(feature):
    return {
        "color": "transparent",
        "fillColor": "transparent"
    }


# Mer komplekse tooltips kan lages ved hjelp av
# `folium.features.GeoJsonTooltip` klassen. Nedenfor bruker vi
# dens mest grunnleggende funksjoner: `fields` spesifiserer hvilke kolonner
# som skal vises, `aliases` hvordan de skal være merket.
tooltip = folium.features.GeoJsonTooltip(
    fields=("",),
    aliases=("",)
)



:::{admonition} Python-pakker for interaktive (web) kart
:class: note

*Folium* er bare en av mange pakker som gir en enkel måte å lage interaktive kart ved hjelp av data lagret i (geo-)pandas dataframer. Andre interessante biblioteker inkluderer:

- [GeoViews](https://geoviews.org/),
- [Mapbox GL for Jupyter](https://github.com/mapbox/mapboxgl-jupyter),
- [Bokeh](https://docs.bokeh.org/en/latest/docs/gallery.html),
- [Plotly Express](https://plotly.com/python/maps/), og mange flere.
:::